# Primjer uzorka agenta za rezervaciju hotela i letova

Ovo rješenje će vam pomoći u rezervaciji avionskih karata i hotela. Scenarij je putovanje iz Londona Heathrow LHR 20. veljače 2024. do New Yorka JFK s povratkom 27. veljače 2025., leteći ekonomskom klasom isključivo s British Airwaysom. Želim boravak u Hilton hotelu u New Yorku, molim vas da mi dostavite cijene za let i hotel.


# Inicijalizirajte Azure AI Agent Service i preuzmite informacije o konfiguraciji iz **.env**

### **.env**

Kreirajte .env datoteku

**.env** sadrži connection string za Azure AI Agent Service, model koji koristi AOAI, te odgovarajući Google API Search servis API, ENDPOINT, itd.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Naziv implementacije modela za Azure AI Agent Service"

[**NOTE**] Trebat će vam model s ograničenjem od 100.000 tokena po minuti (Rate Limit) i ograničenjem od 600 zahtjeva po minuti (Request per minute).

  Model možete dobiti u Azure AI Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Connection string vašeg projekta za Azure AI Agent Service"

  Connection string projekta možete pronaći u pregledu vašeg projekta na AI ​​Foundry Portal ekranu.

- **SERPAPI_SEARCH_API_KEY** = "Vaš SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Vaš SERPAPI Search Endpoint"

Da biste dobili naziv implementacije modela i connection string projekta za Azure AI Agent Service, potrebno je kreirati Azure AI Agent Service. Preporučuje se koristiti [ovaj predložak](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) za direktno kreiranje. (***Napomena:*** Azure AI Agent Service trenutno je dostupan u ograničenim regijama. Preporučuje se da se konzultirate s [ovim linkom](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) kako biste postavili regiju.)

Agent treba pristupiti SERPAPI. Preporučuje se registracija putem [ovog linka](https://serpapi.com/searches). Nakon registracije, možete dobiti jedinstveni API KEY i ENDPOINT.


# Postavljanje

Za pokretanje ovog bilježnika, potrebno je osigurati da ste instalirali potrebne biblioteke pokretanjem `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Vaša verzija Semantic Kernel-a trebala bi biti najmanje 1.27.2.


Učitajte postavke i resurse iz svoje .env datoteke, molimo osigurajte da ste dodali svoje ključeve i postavke te kreirali lokalnu .env datoteku.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Prijava na Azure

Sada se trebate prijaviti na Azure. Otvorite terminal i pokrenite sljedeću naredbu:

```bash
az login
```

Ova naredba će vas zatražiti da unesete svoje Azure vjerodajnice, omogućujući ispravan rad usluge Azure AI Agent.


# Objašnjenje:
Ovo je varijabla koja pohranjuje API ključ za pristup SERP (Search Engine Results Page) API usluzi. API ključ je jedinstveni identifikator koji se koristi za autentifikaciju zahtjeva povezanih s vašim računom.

Svrha: Svrha ovog retka je pohraniti API ključ u varijablu kako bi se mogao koristiti za autentifikaciju zahtjeva prema SERP API usluzi. API ključ je potreban za pristup usluzi i obavljanje pretraga.
Kako dobiti SERP API ključ: Da biste dobili SERP API ključ, slijedite ove opće korake na https://serpapi.com (točni koraci mogu se razlikovati ovisno o specifičnoj SERP API usluzi koju koristite):

Odaberite SERP API uslugu: Dostupno je nekoliko SERP API usluga, poput SerpAPI, Google Custom Search JSON API i drugih. Odaberite onu koja najbolje odgovara vašim potrebama.

Registrirajte se za račun: Idite na web stranicu odabrane SERP API usluge i registrirajte se za račun. Možda ćete trebati pružiti osnovne informacije i potvrditi svoju e-mail adresu.

Kreirajte API ključ: Nakon registracije, prijavite se na svoj račun i idite na odjeljak za API ili nadzornu ploču. Potražite opciju za kreiranje ili generiranje novog API ključa.
Kopirajte API ključ u svoju .env datoteku.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Objašnjenje:
BASE_URL: Ovo je varijabla koja pohranjuje osnovni URL za SERP API krajnju točku. Naziv varijable BASE_URL koristi se kao konvencija za označavanje da je ovaj URL početna točka za slanje API zahtjeva.  
'https://serpapi.com/search':  

Ovo je stvarni URL string koji je dodijeljen varijabli BASE_URL. Predstavlja krajnju točku za izvršavanje upita pretraživanja pomoću SERP API-ja.

# Svrha:
Svrha ovog retka je definirati konstantu koja sadrži osnovni URL za SERP API. Ovaj URL će se koristiti kao početna točka za sastavljanje API zahtjeva za izvršavanje operacija pretraživanja.

# Upotreba:
Definiranjem osnovnog URL-a u varijabli, možete ga lako ponovno koristiti u svom kodu kad god trebate poslati zahtjeve SERP API-ju. Ovo čini vaš kod lakšim za održavanje i smanjuje rizik od pogrešaka koje nastaju tvrdim kodiranjem URL-a na više mjesta. Trenutni primjer je https://serpapi.com/search?engine=bing koji koristi Bing API za pretraživanje. Različiti API-ji mogu se odabrati na https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Objašnjenje:

Ovdje se nalazi vaš kod za dodatak.

Definicija klase: `class BookingPlugin`: Definira klasu pod nazivom BookingPlugin koja sadrži metode za rezervaciju hotela i letova.

Metoda za rezervaciju hotela:

- `@kernel_function(description="booking hotel")`: Dekorator koji opisuje funkciju kao kernel funkciju za rezervaciju hotela.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Definira metodu za rezervaciju hotela s anotiranim parametrima i povratnim tipom.

Metoda konstruira rječnik parametara za zahtjev za rezervaciju hotela i šalje GET zahtjev na SERP API. Provjerava status odgovora i vraća informacije o hotelima ako je uspješno, ili None ako zahtjev nije uspio.

Metoda za rezervaciju letova:

- `@kernel_function(description="booking flight")`: Dekorator koji opisuje funkciju kao kernel funkciju za rezervaciju letova.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Definira metodu za rezervaciju letova s anotiranim parametrima i povratnim tipom.

Metoda konstruira rječnike parametara za zahtjeve za odlazne i povratne letove te šalje GET zahtjeve na SERP API. Provjerava status odgovora i dodaje informacije o letovima u rezultat ako je uspješno, ili ispisuje poruku o pogrešci ako zahtjev nije uspio. Metoda vraća rezultat koji sadrži informacije o letovima.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Objašnjenje:
Importiranje modula: Uvoz potrebnih modula za Azure vjerodajnice, AI agenta, sadržaj poruka u chatu, ulogu autora i dekorator funkcije kernela.

Asinkroni kontekstni upravitelj: `async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,)`: Postavlja asinkroni kontekstni upravitelj za rukovanje Azure vjerodajnicama i kreiranje klijenta AI agenta.

Ime agenta i upute:
- `AGENT_NAME = "BookingAgent"`: Definira ime agenta.
- `AGENT_INSTRUCTIONS = """..."""`: Pruža detaljne upute agentu o tome kako rukovati zahtjevima za rezervaciju.

Kreiranje definicije agenta: `agent_definition = await client.agents.create_agent(...)`: Kreira definiciju agenta s određenim modelom, imenom i uputama.

Kreiranje AzureAI agenta: `agent = AzureAIAgent(...)`: Kreira AzureAI agenta koristeći klijenta, definiciju agenta i definirani dodatak (plugin).

Kreiranje niti: `thread: AzureAIAgentThread | None = None`: Kreira nit za agenta. Nije potrebno prvo kreirati nit - ako je vrijednost `None`, nova nit će biti kreirana tijekom prve invokacije i vraćena kao dio odgovora.

Unosi korisnika: `user_inputs = ["..."]`: Definira popis korisničkih unosa koje agent treba obraditi.

U bloku `finally`, brišu se nit i agent kako bi se oslobodili resursi.


# Autentifikacija

Klasa `DefaultAzureCredential` dio je Azure SDK-a za Python. Pruža zadani način autentifikacije s Azure uslugama. Pokušava se autentificirati koristeći više metoda određenim redoslijedom, poput varijabli okruženja, upravljanog identiteta i vjerodajnica Azure CLI-ja.

Asinkrone operacije: Modul aio označava da klasa DefaultAzureCredential podržava asinkrone operacije. To znači da je možete koristiti s asyncio za izvođenje neblokirajućih zahtjeva za autentifikaciju.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden korištenjem AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane stručnjaka. Ne preuzimamo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja koja mogu proizaći iz korištenja ovog prijevoda.
